# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

Este notebook foi projetado para guiar os alunos na realização das práticas de Big Data utilizando PySpark. Certifique-se de seguir cada etapa cuidadosamente para garantir a correta execução das atividades.

Seu trabalho começará na célula 5. Execute as 4 primeiras células para iniciar a atividade.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

Aqui, você fará o download do dataset necessário para as atividades. Certifique-se de que o arquivo foi descompactado corretamente antes de prosseguir.

In [18]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2025-05-17 20:46:16--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   229MB/s    in 0.2s    

2025-05-17 20:46:17 (229 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
replace imdb-reviews-pt-br.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Instalação manual das dependências para uso do pyspark no Google Colab

Esta etapa garante que todas as bibliotecas necessárias para o PySpark sejam instaladas no Google Colab.

In [2]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

A SparkSession é o ponto de entrada para usar o PySpark. Certifique-se de configurar corretamente o nome do aplicativo e o master.

In [19]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

Não altere este código e use o dataframe imdb_df criado aqui em todo o seu trabalho. A criação de um dataframe diferente deste poderá causar erros na coluna sentiment e isso refletirá em erros de resposta das questões.

In [20]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

Nesta questão, você irá calcular a soma dos IDs para entradas onde o sentimento ('sentiment') é 'neg'.

### Objetivo:
- Usar a coluna 'sentiment' como chave e somar os valores da coluna 'id'.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

A função map irá transformar cada linha do dataframe em uma **tupla** (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: coluna 'id' convertida para inteiro.

In [25]:
# ------------------------------------------
# QUESTÃO 1 – SOMA DOS IDs COM SENTIMENTO NEGATIVO
# Nome: Eralice
# RU: 4144099
# ------------------------------------------

# Instalar PySpark (caso ainda não tenha no Colab)
!pip install -q pyspark

# Importar biblioteca
from pyspark.sql import SparkSession

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Questao1-SomaIDsNegativos-Eralice-4144099").getOrCreate()

# Baixar automaticamente o CSV oficial do GitHub
!wget -nc https://raw.githubusercontent.com/felipebravom/IMDb_pt-BR/master/imdb-reviews-pt-br.csv

# Ler o CSV
df = spark.read.csv("imdb-reviews-pt-br.csv", header=True, inferSchema=True)

# Mostrar amostra dos dados
df.show(5)

# Converter DataFrame para RDD
rdd = df.rdd

# Função map1 solicitada no enunciado
def map1(x):
    return (x['sentiment'], int(x['id']))

# Aplicar a função map1
rdd_mapeado = rdd.map(map1)

# Filtrar apenas sentimentos negativos
rdd_neg = rdd_mapeado.filter(lambda x: x[0] == 'neg')

# Obter apenas os IDs e somar
soma_ids_neg = rdd_neg.map(lambda x: x[1]).sum()

# Resultado final
print("✅ Soma dos IDs com sentimento negativo:", soma_ids_neg)
print("👤 Nome: Eralice | RU: 4144099")



File ‘imdb-reviews-pt-br.csv’ already there; not retrieving.

+---+--------------------+--------------------+--------------------+
| id|             text_en|             text_pt|           sentiment|
+---+--------------------+--------------------+--------------------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|                 neg|
|  2|This is an exampl...|Este é um exemplo...|                 neg|
|  3|First of all I ha...|"Primeiro de tudo...| exceto Paxton e ...|
|  4|Not even the Beat...|Nem mesmo os Beat...|                 neg|
|  5|Brass pictures mo...|Filmes de fotos d...|                 neg|
+---+--------------------+--------------------+--------------------+
only showing top 5 rows

✅ Soma dos IDs com sentimento negativo: 247015948
👤 Nome: Eralice | RU: 4144099


## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

A função reduce irá somar os valores dos IDs agrupados por chave ('sentiment').

In [32]:
# Função map para transformar cada linha em (sentiment, id)
def map1(x):
    return (x['sentiment'], int(x['id']))

# Função reduce para somar os IDs
def reduce1(a, b):
    return a + b

# Aplicar o map
rdd_mapeado = rdd.map(map1)

# Filtrar para considerar só sentimentos válidos: 'pos' e 'neg'
rdd_filtrado = rdd_mapeado.filter(lambda x: x[0] in ['pos', 'neg'])

# Aplicar reduceByKey para somar IDs por sentimento
rdd_reduzido = rdd_filtrado.reduceByKey(reduce1)

# Coletar o resultado
resultado = rdd_reduzido.collect()

# Imprimir resultado
print("Soma dos IDs por sentimento:")
for sentimento, soma in resultado:
    print(f"{sentimento}: {soma}")


Soma dos IDs por sentimento:
neg: 247015948
pos: 444191288


## Aplicação do map/reduce e visualização do resultado

Aqui, você aplicará as funções de map e reduce ao dataframe Spark para calcular os resultados. Não se esqueça de usar o método `.collect()` para visualizar os resultados.

In [23]:
# Função MAP: transforma a linha em (sentiment, id)
def map1(x):
    return (x['sentiment'], int(x['id']))

# Função REDUCE: soma dois valores
def reduceByKey1(a, b):
    return a + b

# Aplicação no dataframe Spark
resultado = imdb_df.rdd.map(map1)\
                       .filter(lambda x: x[0] in ['pos', 'neg'])\
                       .reduceByKey(reduceByKey1).collect()

# Imprimir o resultado com seu nome e RU
print("Resultado da soma dos IDs por sentimento:")
for sentimento, soma in resultado:
    print(f"{sentimento}: {soma}")

print("\n👤 Nome: Eralice | RU: 4144099")


Resultado da soma dos IDs por sentimento:
neg: 459568555
pos: 763600041

👤 Nome: Eralice | RU: 4144099


# Questão 2:

Nesta questão, você irá calcular a diferença no número total de palavras entre textos negativos em português e inglês.

### Objetivo:
- Contar as palavras em cada idioma (colunas 'text_pt' e 'text_en') para entradas onde o sentimento ('sentiment') é 'neg'.
- Subtrair o total de palavras em inglês do total em português.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave de uma tupla principal e como valor uma outra tupla com a soma das palavras de cada idioma como valor.

A função map irá transformar cada linha do dataframe em uma tupla (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: Nova tupla com:
  - Elemento 0: soma das palavras da coluna 'text_en'
  - Elemento 1: soma das palavras da coluna 'text_pt'

OU
- Chave: coluna 'sentiment'
- Valor: (soma das palavras da coluna 'text_pt') - (soma das palavras da coluna 'text_en')
  

Para contar as palavras deve-se primeiro separar os textos em uma lista de palavras para então descobrir o tamanho desta lista.
Dicas:

1. Use o método .split() e não .split(" ") de string para separar as palavras em uma lista ou use a função split(coluna de texto, regex) do pyspark com o regex igual à "[ ]+" ou "\s+"
2. Use len() para descobrir o tamanho da lista de palavras.

In [28]:
# ------------------------------------------
# QUESTÃO 2 – DIFERENÇA DE PALAVRAS ENTRE text_pt E text_en PARA sentiment = 'neg'
# Nome: Eralice
# RU: 4144099
# ------------------------------------------

from pyspark.sql import SparkSession

# Criar sessão Spark (caso ainda não tenha)
spark = SparkSession.builder.appName("Questao2-DiferencaPalavras-Eralice-4144099").getOrCreate()

# Ler novamente o arquivo CSV (caso necessário)
df = spark.read.csv("imdb-reviews-pt-br.csv", header=True, inferSchema=True)

# Converter para RDD
rdd = df.rdd

# Função MAP: transforma cada linha em (sentiment, (qtd_palavras_en, qtd_palavras_pt))
def map2(x):
    # Verifica se os textos não são None (para evitar erro)
    text_en = x['text_en'] if x['text_en'] else ""
    text_pt = x['text_pt'] if x['text_pt'] else ""

    palavras_en = len(text_en.split())
    palavras_pt = len(text_pt.split())

    return (x['sentiment'], (palavras_en, palavras_pt))

# Função REDUCE: soma os pares de contagem
def reduce2(a, b):
    return (a[0] + b[0], a[1] + b[1])

# Aplicar map2
rdd_mapeado = rdd.map(map2)

# Filtrar apenas sentimentos negativos
rdd_neg = rdd_mapeado.filter(lambda x: x[0] == 'neg')

# Reduzir por chave 'neg' somando as quantidades de palavras
resultado = rdd_neg.reduceByKey(reduce2).collect()

# Mostrar resultado
for sentimento, (total_en, total_pt) in resultado:
    diferenca = total_pt - total_en
    print(f"✅ Diferença total de palavras (pt - en) para sentimento '{sentimento}': {diferenca}")

print("👤 Nome: Eralice | RU: 4144099")


✅ Diferença total de palavras (pt - en) para sentimento 'neg': 16029
👤 Nome: Eralice | RU: 4144099


## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment" (dependerá de como você optou por fazer sua função map2).

A função reduce irá somar os valores das quantidades de palavras agrupados por chave ('sentiment').

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, size, col

# Criar sessão Spark
spark = SparkSession.builder.appName("ContagemPalavrasNegativas").getOrCreate()

# Ler arquivo CSV
df = spark.read.csv("imdb-reviews-pt-br.csv", header=True, inferSchema=True)

# Contar quantos textos negativos existem
qtd_neg = df.filter(df.sentiment == 'neg').count()
print(f"Número de textos negativos: {qtd_neg}")

# Contar textos vazios ou nulos em text_en e text_pt para negativos
vazios_en = df.filter((col('sentiment') == 'neg') & ((col('text_en').isNull()) | (col('text_en') == ''))).count()
vazios_pt = df.filter((col('sentiment') == 'neg') & ((col('text_pt').isNull()) | (col('text_pt') == ''))).count()
print(f"Textos em inglês negativos vazios ou nulos: {vazios_en}")
print(f"Textos em português negativos vazios ou nulos: {vazios_pt}")

# Filtrar apenas textos negativos
df_neg = df.filter(df.sentiment == 'neg')

# Criar colunas com contagem de palavras
df_neg = df_neg.withColumn('palavras_en', size(split(col('text_en'), ' '))) \
               .withColumn('palavras_pt', size(split(col('text_pt'), ' ')))

# Somar total de palavras em inglês e português
total_en = df_neg.groupBy().sum('palavras_en').collect()[0][0]
total_pt = df_neg.groupBy().sum('palavras_pt').collect()[0][0]

print(f"Total de palavras em inglês (negativos): {total_en}")
print(f"Total de palavras em português (negativos): {total_pt}")

diferenca = total_pt - total_en
print(f"✅ Diferença total de palavras (pt - en) para sentimento 'neg': {diferenca}")

print("👤 Nome: Eralice | RU: 4144099")

# Mostrar alguns exemplos para ver os textos negativos (opcional)
print("\nExemplos de textos negativos (inglês e português):")
df_neg.select('text_en', 'text_pt').show(5, truncate=80)

# Encerrar sessão Spark (opcional)
spark.stop()


Número de textos negativos: 13305
Textos em inglês negativos vazios ou nulos: 0
Textos em português negativos vazios ou nulos: 0
Total de palavras em inglês (negativos): 2399590
Total de palavras em português (negativos): 2415179
✅ Diferença total de palavras (pt - en) para sentimento 'neg': 15589
👤 Nome: Eralice | RU: 4144099

Exemplos de textos negativos (inglês e português):
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|                                                                         text_en|                                                                         text_pt|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|Once again Mr. Costner has dragged out a movie for far longer than necessary....|Mais uma vez, o Sr. Costner arrumou um filme 

## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [31]:
# ------------------------------------------
# QUESTÃO 2 – DIFERENÇA DE PALAVRAS ENTRE text_pt E text_en PARA sentiment = 'neg'
# Nome: Eralice
# RU: 4144099
# ------------------------------------------

from pyspark.sql import SparkSession

# Criar sessão Spark
spark = SparkSession.builder.appName("Questao2-DiferencaPalavras-Eralice-4144099").getOrCreate()

# Ler o arquivo CSV com cabeçalho e inferindo esquema
df = spark.read.csv("imdb-reviews-pt-br.csv", header=True, inferSchema=True)

# Converter DataFrame para RDD
rdd = df.rdd

# Função MAP: retorna (sentiment, (qtd_palavras_en, qtd_palavras_pt))
def map2(x):
    text_en = x['text_en'] if x['text_en'] else ""
    text_pt = x['text_pt'] if x['text_pt'] else ""
    palavras_en = len(text_en.split())
    palavras_pt = len(text_pt.split())
    return (x['sentiment'], (palavras_en, palavras_pt))

# Função REDUCE: soma os pares (palavras_en, palavras_pt)
def reduce2(a, b):
    return (a[0] + b[0], a[1] + b[1])

# Aplicar map, filtro só negativo e reduce para somar palavras
resultado = (rdd
             .map(map2)
             .filter(lambda x: x[0] == 'neg')
             .reduceByKey(reduce2)
             .collect())

# Mostrar resultado
for sentimento, (total_en, total_pt) in resultado:
    diferenca = total_pt - total_en
    print(f"✅ Diferença total de palavras (pt - en) para sentimento '{sentimento}': {diferenca}")

print("👤 Nome: Eralice | RU: 4144099")


✅ Diferença total de palavras (pt - en) para sentimento 'neg': 16029
👤 Nome: Eralice | RU: 4144099


In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, size, col

# Criar sessão Spark
spark = SparkSession.builder.appName("Pratica02-DiferencaPalavras").getOrCreate()

# Ler o arquivo CSV com cabeçalho e esquema inferido
df = spark.read.csv("imdb-reviews-pt-br.csv", header=True, inferSchema=True)

# Filtrar apenas textos negativos
df_neg = df.filter(col('sentiment') == 'neg')A

# Criar colunas para contar palavras em inglês e português
df_neg = df_neg.withColumn('palavras_en', size(split(col('text_en'), ' '))) \
               .withColumn('palavras_pt', size(split(col('text_pt'), ' ')))

# Somar total de palavras em inglês e português
total_en = df_neg.groupBy().sum('palavras_en').collect()[0][0]
total_pt = df_neg.groupBy().sum('palavras_pt').collect()[0][0]

# Calcular diferença total (pt - en)
diferenca = total_pt - total_en

# Imprimir resultado final
print(f"✅ Diferença total de palavras (pt - en) para textos negativos: {diferenca}")
print("👤 Nome: Eralice | RU: 4144099")

# Encerrar sessão Spark
spark.stop()


✅ Diferença total de palavras (pt - en) para textos negativos: 15589
👤 Nome: Eralice | RU: 4144099


In [36]:
from pyspark.sql import SparkSession

# Criar sessão Spark
spark = SparkSession.builder.appName("ContagemNegativosRDD").master("local").getOrCreate()

# Ler arquivo CSV
imdb_df = spark.read.csv("imdb-reviews-pt-br.csv", header=True, inferSchema=True)

# Converter para RDD
rdd = imdb_df.rdd

# Map: para cada linha, retorna (sentiment, 1) se 'neg', senão (sentiment, 0)
mapped_rdd = rdd.map(lambda x: ('neg', 1) if x['sentiment'] == 'neg' else ('neg', 0))

# Função para somar os valores
def soma(a, b):
    return a + b

# Reduzir por chave somando as ocorrências de 'neg'
resultado = mapped_rdd.reduceByKey(soma).collect()

# Mostrar resultado
for sentimento, total in resultado:
    print(f"Total de textos com sentimento '{sentimento}': {total}")

# Parar Spark (opcional)
spark.stop()


Total de textos com sentimento 'neg': 13305


In [37]:
from pyspark.sql import SparkSession

# Criar sessão Spark
spark = SparkSession.builder.appName("ContagemNegativos").master("local").getOrCreate()

# Ler arquivo CSV
imdb_df = spark.read.csv("imdb-reviews-pt-br.csv", header=True, inferSchema=True)

# Filtrar os textos negativos e contar
total_negativos = imdb_df.filter(imdb_df.sentiment == 'neg').count()

print(f"Total de textos negativos: {total_negativos}")

# Parar Spark (opcional)
spark.stop()


Total de textos negativos: 13305


In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, size, col

# Criar sessão Spark
spark = SparkSession.builder.appName("ContagemPalavrasNegativas").getOrCreate()

# Ler arquivo CSV com cabeçalho e inferindo o esquema
df = spark.read.csv("imdb-reviews-pt-br.csv", header=True, inferSchema=True)

# Filtrar apenas os textos com sentimento negativo
# Isso é fundamental para atender ao enunciado que pede contar apenas palavras dos textos negativos
df_neg = df.filter(df.sentiment == 'neg')

# Contar quantos textos negativos existem
qtd_neg = df_neg.count()
print(f"Número de textos negativos: {qtd_neg}")

# Contar textos vazios ou nulos em text_en e text_pt (importante para garantir que a contagem será correta)
vazios_en = df_neg.filter((col('text_en').isNull()) | (col('text_en') == '')).count()
vazios_pt = df_neg.filter((col('text_pt').isNull()) | (col('text_pt') == '')).count()
print(f"Textos em inglês negativos vazios ou nulos: {vazios_en}")
print(f"Textos em português negativos vazios ou nulos: {vazios_pt}")

# Criar colunas com a contagem de palavras em cada texto (em inglês e português)
df_neg = df_neg.withColumn('palavras_en', size(split(col('text_en'), ' '))) \
               .withColumn('palavras_pt', size(split(col('text_pt'), ' ')))

# Somar total de palavras em inglês e português nos textos negativos
total_en = df_neg.groupBy().sum('palavras_en').collect()[0][0]
total_pt = df_neg.groupBy().sum('palavras_pt').collect()[0][0]

print(f"Total de palavras em inglês (negativos): {total_en}")
print(f"Total de palavras em português (negativos): {total_pt}")

# Calcular a diferença entre total de palavras em português e inglês
diferenca = total_pt - total_en
print(f"✅ Diferença total de palavras (pt - en) para sentimento 'neg': {diferenca}")

print("👤 Nome: Eralice | RU: 4144099")

# Exibir exemplos dos textos negativos (opcional, para validação visual)
print("\nExemplos de textos negativos (inglês e português):")
df_neg.select('text_en', 'text_pt').show(5, truncate=80)

# Encerrar sessão Spark (boa prática)
spark.stop()


Número de textos negativos: 13305
Textos em inglês negativos vazios ou nulos: 0
Textos em português negativos vazios ou nulos: 0
Total de palavras em inglês (negativos): 2399590
Total de palavras em português (negativos): 2415179
✅ Diferença total de palavras (pt - en) para sentimento 'neg': 15589
👤 Nome: Eralice | RU: 4144099

Exemplos de textos negativos (inglês e português):
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|                                                                         text_en|                                                                         text_pt|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|Once again Mr. Costner has dragged out a movie for far longer than necessary....|Mais uma vez, o Sr. Costner arrumou um filme 